In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)

config = load_config()
conn = connect(config)
conn.autocommit = True
cursor = conn.cursor()

Connected to the PostgreSQL server.


In [ ]:
cursor.execute("CREATE DATABASE health")

In [4]:
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS activity (
        _id SERIAL PRIMARY KEY,
        date DATE NOT NULL,
        time TIME NOT NULL,
        activity BOOL,
        acceleration_x FLOAT NOT NULL,
        acceleration_y FLOAT NOT NULL,
        acceleration_z FLOAT NOT NULL,
        gyro_x FLOAT NOT NULL,
        gyro_y FLOAT NOT NULL,
        gyro_z FLOAT NOT NULL
    )
    """)

In [ ]:
file_path = '../training/training.csv'
training_df = pd.read_csv(file_path, sep=';')
training_df.head()

# # Convert 'date' to datetime (format: YYYY-MM-DD)
# training_df['date'] = pd.to_datetime(training_df['date'], format='%d/%m/%Y', errors='coerce')
# # Convert 'time' to time format
# training_df['time'] = pd.to_datetime(training_df['time'], format='%H:%M:%S', errors='coerce').dt.time
# # Convert 'activity' to boolean
# training_df['activity'] = training_df['activity'].astype(bool)
# Convert acceleration and gyroscope columns to float
float_columns = ['acceleration_x', 'acceleration_y', 'acceleration_z', 
                 'gyro_x', 'gyro_y', 'gyro_z']
for col in float_columns:
    training_df[col] = pd.to_numeric(training_df[col], errors='coerce')

In [27]:
# Convert DataFrame rows to a list of tuples
data = training_df.to_records(index=False)  # Exclude index for insertion
records = [tuple(row) for row in data]

# Batch insert into the activity table
query = """
    INSERT INTO activity (date, time, activity, acceleration_x, acceleration_y, acceleration_z, gyro_x, gyro_y, gyro_z)
    VALUES %s
"""
execute_values(cursor, query, records)

ProgrammingError: can't adapt type 'numpy.int64'